## _Change log_
1. Using a U-net trained on a much larger dataset of free swimming images. Assessment revealed great performance.
   

In [ ]:
import os, random, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import tifffile as tff
import dask
from dask.diagnostics import ProgressBar
import joblib
import seaborn as sns
import glob

codeDir = r'\\dm11\koyamalab/code/python/code'
sys.path.append(codeDir)

import apCode.FileTools as ft
import apCode.volTools as volt
import apCode.behavior.FreeSwimBehavior as fsb
import apCode.behavior.headFixed as hf
import apCode.SignalProcessingTools as spt
from apCode import util as util
import rsNeuronsProj.util as rsp
from apCode.machineLearning import ml as mlearn

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

try:
    if __IPYTHON__:
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

# Setting seed for reproducability
seed = 143
random.seed = seed

print(time.ctime())

### *Load dataframe containing paths to data*

In [ ]:
#%% Path to excel sheet storing paths to data and other relevant info
dir_df = r'Y:\Avinash\Projects\RS recruitment\Ablations'
path_df = glob.glob(os.path.join(dir_df, 'Ablation data summary*.pkl'))[-1]

df = pd.read_pickle(path_df)
dir_save = os.path.join(dir_df, f'session_{util.timestamp()}')
os.makedirs(dir_save, exist_ok=True)
df_orig = df.copy()
df.head()

### *Load pretrained U-net (loss_func = focal_loss; optimizer='Adam')*

In [ ]:
dir_unet = r'Y:\Avinash\Ablations and Behavior'
file_unet = ft.findAndSortFilesInDir(dir_unet, ext='h5', search_str='trainedU_fsb_896x896_2020')[-1]
print(f'Loading U-net: {file_unet}')
path_unet = os.path.join(dir_unet, file_unet)

unet = mlearn.loadPreTrainedUnet(path_unet)
print(f'U net dims = {unet.input_shape}')
print(time.ctime())

### *Track fish in each of the datasets and create a larger dataframe that also contains tail angles info*

In [ ]:
df.columns

In [ ]:
%%time
#%% Create another dataframe with tail angles info and merge with original dataframe using common FishIdx column
trlLen = 750
df = df_orig.loc[df_orig.Illumination=='diffuse']
motion_thresh_perc=None

paths_retrack = []
nFish = df.shape[0]
trackedWithNN = np.zeros_like(df.TrackedWithNN)
hFilePaths = []
for iPath in range(df.shape[0]):
    print(f'{iPath+1}/{nFish}')
    df_ = df.iloc[iPath]
    path_imgs = rsp.remove_suffix_from_paths(df_.Path, suffix='proc')[()]
    try:
        hFilePath = fsb.tail_angles_from_raw_imgs_using_unet(path_imgs, unet,\
                                                             motion_thresh_perc=motion_thresh_perc)
        trackedWithNN[iPath]=1
        hFilePaths.append(hFilePath)
    except Exception:
        print('Tracking failed!')
    
    

In [ ]:
len(hFilePaths), len(df), df.columns

In [ ]:
df.iloc[0].Path_proc

In [ ]:
# fsb.tail_angles_from_raw_imgs_using_unet??

## _Filter dataframe based on quality of tracking, then interpolate tail angles for rest_

In [ ]:
%%time
thr_track = 85
trlLen=750

df = dataFrame_orig.loc[dataFrame_orig.perc_tailAngles_tracked>=thr_track]
ta_full=[]
for iFish in range(df.shape[0]):    
    df_ = df.iloc[iFish]
#     print(f'Fish # {iFish+1}/{df.shape[0]}, perc_tracked = {df_.perc_tailAngles_tracked} %')
    ta_ = df_.tailAngles
    inds_ta = df_.inds_tailAngles
    nTot = np.maximum(df_.totalNumPts, inds_ta.max()+1)
    ta_nan = np.zeros((ta_.shape[0], nTot), dtype='float')*np.nan
    ta_nan[:, inds_ta] = ta_
#     print('Interpolating...')
#     taf = spt.interp.nanInterp2d(ta_nan, method='nearest')
    taf = dask.delayed(spt.interp.nanInterp2d)(ta_nan, method='nearest')
    ta_full.append(taf)
with ProgressBar():
    ta_full = dask.compute(*ta_full, scheduler='processes')
ta_full_clip, ta_tot = [], []
for ta_ in ta_full:
    nTrls = ta_.shape[1]//trlLen
    n = nTrls*trlLen
    ta_ = ta_[:, :n]
    ta_full_clip.append(ta_)
    ta_tot.append(ta_[-1])
ta_full = ta_full_clip
df = df.assign(tailAngles=ta_full_clip, tailAngles_tot=ta_tot)    

## _Clean up tail angles in the dataframe_

In [ ]:
%%time
dt_behav = 1/500
nFish = len(ta_full)
ta_full_ser = np.concatenate(ta_full, axis=1)
%time ta_ser_clean, _, svd = hf.cleanTailAngles(ta_full_ser, dt=dt_behav)

tLens = np.cumsum(np.array([ta_.shape[1] for ta_ in ta_full]))
ta_full_clean = np.hsplit(ta_ser_clean, tLens)
ta_full_clean.pop()

ta_tot = []
for ta_ in ta_full_clean:
    ta_tot.append(ta_[-1])
df = df.assign(tailAngles=ta_full_clean, tailAngles_tot=ta_tot)
ta_full = ta_full_clean

## _Save the dataframe for future use_

In [ ]:
fname = f'dataFrame_rsNeurons_ablations_svdClean_{util.timestamp()}.pkl'
%time df.to_pickle(os.path.join(dir_save, fname))
print(f'Saved at\n{dir_save}')

In [ ]:
iFish = (0, 0)
xl = (-20, 320) # ms
yl = (-200, 200)
alpha = 0.3

inds_ctrl = np.where((df.AblationGroup=='ventralRS') & (df.Treatment=='ctrl'))[0]
inds_abl = np.where((df.AblationGroup=='ventralRS') & (df.Treatment=='abl'))[0]

fh, ax = plt.subplots(3, 1, figsize=(20, 12), sharex=True, sharey=False)

y = ta_tot[inds_ctrl[iFish[0]]]*1.3
y = spt.chebFilt(y, 1/500, 5, btype='highpass')
y_trl = y.reshape(-1, trlLen)
t = (np.arange(y_trl.shape[1])-50)*(1/500)*1000
if len(y_trl)<25:
    y_trl = util.CombineItems(N=25).fit(y_trl).transform(y_trl)

ax[0].plot(t, y_trl.T, alpha=alpha)
ax[0].set_title('$Control$', fontsize=16)
ax[1].set_ylim(yl)

env = spt.emd.envelopesAndImf(y)['env']['diff']
env = env.reshape(-1, trlLen)
env = env-env[:, :40].mean(axis=1)[:, None]
mu = env.mean(axis=0)
sem = env.std(axis=0)/env.shape[0]**0.5
ax[2].fill_between(t, mu-sem, mu+sem, color=plt.cm.tab10(0), alpha=0.5, label='Control')
ax[2].set_ylim(-20, yl[1]*0.55)

y = ta_tot[inds_abl[iFish[1]]]
y = spt.chebFilt(y, 1/500, 10, btype='highpass')
y_trl = y.reshape(-1, trlLen)
if len(y_trl)<25:
    y_trl = util.CombineItems(N=25).fit(y_trl).transform(y_trl)
    
   
    
ax[1].plot(t, y_trl.T, alpha=alpha)
ax[1].set_title('$Ablated$', fontsize=16)
ax[1].set_xlim(xl)
ax[1].set_ylim(yl)
ax[1].set_ylabel('$Bend \ amp (^o) $', fontsize=16)

env = spt.emd.envelopesAndImf(y)['env']['diff']
env = env.reshape(-1, trlLen)
env = env-env[:, :40].mean(axis=1)[:, None]
mu = env.mean(axis=0)
sem = env.std(axis=0)/env.shape[0]**0.5
ax[2].fill_between(t, mu-sem, mu+sem, color=plt.cm.tab10(1), alpha=0.5, label='Control')
ax[2].set_xlabel('$Time\ (ms)$', fontsize=16)
ax[2].set_ylim(-20, yl[1]*0.55)
ax[2].legend(fontsize=16)
ax[2].set_title('$Envelopes (\mu \pm \sigma/\sqrt{n})$', fontsize=16)

fh.suptitle('Ablation Group: Ventral RS', fontsize=20)

# for ext in ['pdf', 'png']:
#     fh.savefig(os.path.join(dir_save, f'Fig-{util.timestamp("day")}_ventral RS_ctrl-vs-abl' + f'.{ext}'), dpi='figure')

In [ ]:
#%% Load the trained model
dir_group = r'Y:\Avinash\Projects\RS recruitment\GCaMP imaging\Group'
# file_model = 'gmm_svd-3_env_pca-9_gmm-20_20200129-18.pkl'
# path_gmm_model = os.path.join(dir_group, file_model)
path_gmm_model = glob.glob(os.path.join(dir_group, 'gmm_svd-3_env_pca-9_gmm-20*.pkl'))[-1]
gmm_model = joblib.load(path_gmm_model)


In [ ]:
iTrl = 10
y = ta_full[inds_abl[iFish[0]]]
y_trl = np.array(np.hsplit(y, y.shape[1]/trlLen))
lbls = gmm_model.predict(y_trl[iTrl])[0]
plt.figure(figsize=(20, 5))
plt.plot(t, y_trl[iTrl][-1])
for lbl in np.unique(lbls):
    inds = np.where(lbls==lbl)[0]
    plt.scatter(t[inds][::4], y_trl[iTrl][-1][inds][::4], c=np.array(plt.cm.tab20(lbl))[None, :], marker=f'${str(lbl)}$', s=200)
plt.xlim(-20, 700)
y_trl.shape
plt.figure(figsize=(20, 5))
plt.pcolor(t, range(y_trl.shape[1]), np.flipud(y_trl[iTrl]))

In [ ]:
a = np.random.randint(0, 10, 20)
_, inds = np.unique(a, return_index=True)


In [ ]:
y = ta_trl[6][-1]
def envelopes_allPks(x, triplicate: bool = False, interp_kind: str = 'slinear'):
    from apCode.SignalProcessingTools import timeseries as ts
    from apCode.SignalProcessingTools import findPeaks
    from scipy.interpolate import interp1d
    if triplicate:
        x = ts.triplicate(x)
        indVec = np.arange(len(x))
    pks = findPeaks(x, pol=0)[0]
    xp = np.hstack((0, pks.ravel(), len(x)))
    _, inds = np.unique(xp, return_index=True)
    xp = xp[inds]
    fp = np.hstack((x[0], x[pks], x[-1]))
    fp = np.abs(fp[inds])
    xx = np.arange(0, len(x))
    f = interp1d(xp, fp, kind=interp_kind)
    env = f(xx)
    if triplicate:
        env = ts.middleThird(env)
    return env
env = spt.emd.envelopesAndImf(y, triplicate=True)['env']   
env_diff = env['diff']
env_max = env['max']
env_abs = envelopes_allPks(y, triplicate=True)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(y, label='Original signal')
plt.plot(env_diff, label=f"Envelopes: Avi's method")
# plt.plot(env_max)
plt.plot(env_abs, label="Envelopes: Minoru's method")
plt.legend(loc='best', fontsize=16)
plt.xlim(0, 350)
plt.savefig(os.path.join(dir_save, f'Fig-{util.timestamp("second")}_Comparison of envelopes.png'), dpi='figure')


## _Reload dataframe to continue from here_

In [ ]:
fname = glob.glob(os.path.join(dir_xls, 'dataFrame_rsNeurons_ablations_clean_*.pkl'))[-1]
dataFrame_orig = pd.read_pickle(fname)


In [ ]:
dir_xls

## _Load trained GMM model and predict on tail angles from dataframe_

In [ ]:
#%% Load the trained model
dir_group = r'Y:\Avinash\Projects\RS recruitment\GCaMP imaging\Group'
# file_model = 'gmm_svd-3_env_pca-9_gmm-20_20200129-18.pkl'
# path_gmm_model = os.path.join(dir_group, file_model)
path_gmm_model = glob.glob(os.path.join(dir_group, 'gmm_svd-3_env_pca-9_gmm-20*.pkl'))[-1]
gmm_model = joblib.load(path_gmm_model)


In [ ]:
#%% Predict labels using GMM
Fs=500

ta_trl = [np.array(_) for _ in df.tailAngles]
ta = np.concatenate(ta_trl, axis=1)
ta_tot = spt.chebFilt(ta[-1], 1/Fs, (5, 60), btype='bandpass')
fishIdx_ser = [np.ones((ta_.shape[1],))*fi for fi, ta_ in zip(df.FishIdx, df.tailAngles)]
fishIdx_ser = np.concatenate(fishIdx_ser, axis=0).astype(int)
%time labels, features = gmm_model.predict(ta)
%time envelopes = spt.emd.envelopesAndImf(ta_tot)['env']
env_max, env_diff = envelopes['max'], envelopes['diff']

In [ ]:
#%% Plot to check out what the labels look like
xl = (10e3, 15e3)

inds = np.arange(*xl, dtype='int')
plt.figure(figsize=(20, 5))
# plt.plot(ta_clean[-1])
x = np.arange(len(inds))
# plt.plot(x, ta_tot[inds])
plt.plot(x, env_diff[inds])
clrs = plt.cm.tab20(labels[inds])
for x_ in x[::10]:
    plt.scatter(x_, env_diff[inds][x_], c=clrs[x_][None, :], marker=r'${}$'.format(labels[inds][x_]), s=200)
plt.xlim(0, len(inds))
# plt.xlim(2000, 3e3)
plt.ylim(-10, 100)

## _Detection of swim onsets and offsets followed by modificaton of dataframe to include episodes of tail angles_

In [ ]:
%%time
tPre=25e-3
tPost = 100e-3
thr_bend=8
Fs = 500

print('Computing swim on- and offsets...')
ons, offs, _ = fsb.swimOnAndOffsets(ta_tot)
ons = ons-int(tPre*Fs)
offs = offs+int(tPost*Fs)

if len(ons)!=len(offs):
    raise Exception("Onsets and offsets don't match")

# Remove episodes too close to beginning or end
inds_del = np.where(ons<0)[0]
ons = np.delete(ons, inds_del, axis=0)
offs = np.delete(offs, inds_del, axis=0)
inds_el = np.where(offs>(len(ta_tot)-1))[0]
ons = np.delete(ons, inds_del, axis=0)
offs = np.delete(offs, inds_del, axis=0)

# Episodic tail angles to be incorporated into the data
ta_tot_ep, ta_ep, env_diff_ep, env_max_ep = [], [], [], []
bendNum, bendAmp, bendInt =[], [], []
fishIdx_ep = []
for on, off in zip(ons, offs):
    y=ta_tot[on:off]
    pks = spt.findPeaks(y, pol=0, thr=thr_bend, thrType='rel')[0]
    if len(pks)>1:
        nBends = len(pks)
        bendNum.append(np.arange(nBends))
        bendAmp.append(y[pks])
        bendInt.append(np.round(np.gradient(pks)*(1000/Fs)))
    else:
        bendNum.append([np.nan])
        bendAmp.append([np.nan])
        bendInt.append([np.nan])
    ta_tot_ep.append(y)
    ta_ep.append(ta[:,on:off])
    env_diff_ep.append(env_diff[on:off])
    env_max_ep.append(env_max[on:off])
    fishIdx_ep.append(int(fishIdx_ser[on]))
dict_list = dict(tailAngles=ta_ep, tailAngles_tot=ta_tot_ep, tailAngles_env_max=env_max_ep,
                 tailAngles_env_diff=env_diff_ep, FishIdx=fishIdx_ep,
                 episodeNum_glob=list(range(len(ons))), bendNum=bendNum,
                 bendAmp=bendAmp, bendInt=bendInt)
df_now = pd.DataFrame(dict_list)

dataFrame_drop = dataFrame_orig.drop(columns=['tailAngles', 'tailAngles_tot'])
dataFrame = pd.merge(dataFrame_drop, df_now, on='FishIdx')
print(dataFrame.columns)

In [ ]:
dataFrame.head()

## *Expand dataframe on bends*

In [ ]:
dic = dict(episodeNum_glob=[], bendNum=[], bendAmp=[], bendInt=[], nBends=[])
for iEp in range(dataFrame.shape[0]):
    ep = dataFrame.iloc[iEp]
    bendNum = np.array(ep.bendNum)
    epNum = ep.episodeNum_glob*np.ones_like(bendNum)
    nBends = len(bendNum)*np.ones_like(bendNum)
    dic['episodeNum_glob'].extend(epNum)
    dic['bendNum'].extend(bendNum)
    dic['bendAmp'].extend(np.array(ep.bendAmp))
    dic['bendInt'].extend(np.array(ep.bendInt))  
    dic['nBends'].extend(nBends)
df_now = pd.DataFrame(dic)
df_old = dataFrame.drop(columns=['bendNum', 'bendAmp', 'bendInt'])
dataFrame = pd.merge(df_old, df_now, on='episodeNum_glob')

## _Save dataframe for later use_

In [ ]:
fname = f'dataFrame_rsNeurons_ablations_clean_bends_{util.timestamp()}.pkl'
%time dataFrame.to_pickle(os.path.join(dir_xls, fname))

In [ ]:
#%% Flippity do
grps = np.array(dataFrame.AblationGroup)
trts = np.array(dataFrame.Treatment)
inds_ctrl = np.where((grps=='ventralRS') & (trts=='ctrl'))[0]
inds_abl = np.where((grps=='ventralRS') & (trts=='abl'))[0]
trts[inds_ctrl] = 'abl'
trts[inds_abl] ='ctrl'
dataFrame_flp = dataFrame.copy()
dataFrame_flp = dataFrame_flp.assign(Treatment=trts)

fname = f'dataFrame_rsNeurons_ablations_clean_bends_venFlp_{util.timestamp()}.pkl'
%time dataFrame_flp.to_pickle(os.path.join(dir_xls, fname))

## _Reload dataframe to continue from here_

In [ ]:
fname = glob.glob(os.path.join(dir_xls, 'dataFrame_rsNeurons_ablations_clean_bends_venFlp*.pkl'))[-1]
dataFrame = pd.read_pickle(fname)
dataFrame_orig = dataFrame.copy()

## _First, just look at number of bends per episode for each ablation group, comparing ctrl vs abl_

In [ ]:
dataFrame.columns

In [ ]:
nBends_max = 40
nBends_min = 5
df_sub = dataFrame.loc[(dataFrame.bendNum<=nBends_max) & (dataFrame.nBends<=nBends_max)
                       & (dataFrame.nBends>=nBends_min)]
sns.catplot(data=df_sub, x='AblationGroup', y='nBends', hue='Treatment', kind='boxen',
            aspect=2, order= ['mHom', 'intermediateRS', 'ventralRS'], hue_order=['ctrl', 'abl'])

In [ ]:
nBends_max = 35

plt.figure(figsize=(10, 15))
ablGrps = ['mHom', 'intermediateRS', 'ventralRS']
nGrps = len(ablGrps)
bendNums = np.arange(nBends_max)
for iGrp, ablGrp in enumerate(ablGrps):
    grp = df_sub.loc[df_sub.AblationGroup==ablGrp]
    trts = ['ctrl', 'abl']
    p_ctrl =[]
    p_abl = []
    for iTrt, trt in enumerate(trts):
        foo = grp.loc[grp.Treatment==trt]
        nEps = len(foo.loc[foo.bendNum==0])
        for bn in bendNums:
            n = len(foo.loc[foo.bendNum==bn])
            p = n/nEps
            if trt == 'ctrl':
                p_ctrl.append(p)
            else:
                p_abl.append(p)
    p_ctrl, p_abl = np.array(p_ctrl), np.array(p_abl)     
    plt.subplot(nGrps, 1, iGrp+1)
    plt.plot(bendNums+1, p_ctrl,'o-', label='Control', alpha=0.5)
    plt.plot(bendNums+1, p_abl,'o-', label = 'Ablated', alpha=0.5)
    plt.xlim(0, bendNums.max())
    plt.title(ablGrp, fontsize=14)
    if iGrp==0:
        plt.legend()
    plt.ylabel('$P(bend)$')
plt.xlabel('Bend #')
plt.subplots_adjust(hspace=0.3)


In [ ]:
dataFrame

In [ ]:
#%% Create a dataframe that includes swimming information (fish position, swim distance, etc)
%time df = rsp.append_fishPos_to_xls(xls)

#%% Compute onset latencies
%time df = rsp.detect_noisy_trials(df)

df_den = df.loc[df.noisyTrlInds_swimVel==0]
df = rsp.append_latency_to_df(df_den, zThr=0.5)
inds_del = np.where(np.isnan(df.onsetLatency))[0]
df = df.set_index(np.arange(df.shape[0]))
df = df.drop(index=inds_del)
df = df.assign(onsetLatency_log=np.log(df.onsetLatency))


## Adjust swim distances and velocities to make consistent across ablation groups

In [ ]:
sf_dist = 1.25
sf_vel = 1.4

inds = np.where(df.AblationGroup != 'mHom')[0]
swimDist_total = np.array(df.swimDist_total)
swimDist_total[inds] = swimDist_total[inds]*sf_dist

swimVel_max = np.array(df.swimVel_max)
swimVel_max[inds] = swimVel_max[inds]*sf_vel


df = df.assign(swimDist_total_adj=swimDist_total,\
               swimDist_total_adj_log=np.log2(swimDist_total),
               swimVel_max_adj=swimVel_max,\
               swimVel_max_adj_log=np.log2(swimVel_max))


# Categorical plots

In [ ]:
#%% Total swim distance
saveDir = f'Y:\Avinash\Projects\RS recruitment\Figures\{util.timestamp("day")}'
#%% Filter extreme values
q_max = 99
q_min = 5

if not os.path.exists(saveDir):
    os.mkdir(saveDir)
thr_low = np.percentile(df.swimDist_total, q_min)
thr_high = np.percentile(df.swimDist_total, q_max)

df_sub = df.loc[(df.swimDist_total > thr_low) & (df.swimDist_total < thr_high)]


figName = f'Fig_{util.timestamp()}_Total swim distance_ctrl-vs-abl_mHom_interRS_ventralRS'
fh = sns.catplot(data= df_sub, x = 'AblationGroup', y = 'swimDist_total_adj',
                 kind = 'boxen', sharey=True, hue = 'Treatment', dodge = True,
                 order = ['mHom', 'intermediateRS', 'ventralRS'],
                 aspect=2)
fh.savefig(os.path.join(saveDir, figName + '_boxen.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(saveDir, figName + '_boxen.png'), format = 'png', dpi = 'figure')

figName = f'Fig_{util.timestamp()}_Total swim distance_log_ctrl-vs-abl_mHom_interRS_ventralRS'
fh = sns.catplot(data= df_sub, x = 'AblationGroup', y = 'swimDist_total_adj_log',
                 kind = 'boxen', sharey=True, hue = 'Treatment', dodge = True,
                 order = ['mHom', 'intermediateRS', 'ventralRS'],
                 aspect=2)
fh.savefig(os.path.join(saveDir, figName + '_boxen.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(saveDir, figName + '_boxen.png'), format = 'png', dpi = 'figure')



plt.show()
thr_low = np.percentile(df.swimVel_max, q_min)
thr_high = np.percentile(df.swimVel_max, q_max)

df_sub = df.loc[(df.swimVel_max > thr_low) & (df.swimVel_max < thr_high)]

figName = f'Fig_{util.timestamp()}_Max swim vel_ctrl-vs-abl_mHom-interRS-ventralRS_boxen'
fh = sns.catplot(data= df_sub, x = 'AblationGroup', y = 'swimVel_max_adj',
                 kind = 'boxen', sharey=True, hue = 'Treatment',\
                 dodge = True, order = ['mHom', 'intermediateRS', 'ventralRS'],
                 aspect=2)
fh.savefig(os.path.join(saveDir, figName + '_boxen.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(saveDir, figName + '_boxen.png'), format = 'png', dpi = 'figure')


figName = f'Fig_{util.timestamp()}_Max swim vel_log_ctrl-vs-abl_mHom-interRS-ventralRS'
fh = sns.catplot(data= df_sub, x = 'AblationGroup', y = 'swimVel_max_adj_log',
                 kind = 'boxen', sharey=True, hue = 'Treatment',\
                 dodge = True, order = ['mHom', 'intermediateRS', 'ventralRS'],
                 aspect=2)
fh.savefig(os.path.join(saveDir, figName + '_boxen.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(saveDir, figName + '_boxen.png'), format = 'png', dpi = 'figure')



In [ ]:

#%% Plot latencies
minLat = 5
maxLat = 30

    
fn = f'Fig-{util.timestamp()}_Onset latencies_log_ctrl-vs-abl_mHom_interRS_ventralRS'
df_now = df.loc[(df.onsetLatency >= minLat) & (df.onsetLatency <= maxLat)]

fh = sns.catplot(data= df_now, x='AblationGroup', y='onsetLatency_log', hue='Treatment',
                 order=['mHom', 'intermediateRS', 'ventralRS'],
                 margin_titles=True, kind='boxen', dodge=True, aspect=2)
fh.savefig(os.path.join(saveDir, fn + '_boxen.pdf'))
fh.savefig(os.path.join(saveDir, fn + '_boxen.png'))

plt.show()

fh = sns.catplot(data= df_now, x='AblationGroup', y='onsetLatency_log', hue='Treatment',
                 order=['mHom', 'intermediateRS', 'ventralRS'],
                 margin_titles=True, kind='swarm', dodge=True, aspect=2, alpha=0.5)
fh.savefig(os.path.join(saveDir, fn + '_swarm.pdf'))
fh.savefig(os.path.join(saveDir, fn + '_swarm.png'))

plt.show()
fh = sns.catplot(data= df_now, x='AblationGroup', y='onsetLatency_log', hue='Treatment',
                 order=['mHom', 'intermediateRS', 'ventralRS'],
                 margin_titles=True, kind='box', dodge=True, aspect=2)
fh.savefig(os.path.join(saveDir, fn + '_box.pdf'))
fh.savefig(os.path.join(saveDir, fn + '_box.png'))

# Save to CSV/pickle file file

In [ ]:
%%time
saveDir = r'Y:\Avinash\Projects\RS recruitment\Ablations\CSV files'
fn = f'Dataframe with swim trajectories and global swim parameters_{util.timestamp()}'
%time df.to_csv(os.path.join(saveDir,fn + '.csv'), columns=df.keys(), index = False)
%time df.to_pickle(os.path.join(saveDir, fn + '.pkl'))

## Resume analysis

In [ ]:
dir_csv = r'Y:\Avinash\Projects\RS recruitment\Ablations\CSV files'
fn = ft.findAndSortFilesInDir(dir_csv, search_str='Dataframe with swim trajectories', ext = '.pkl')[-1]
df = pd.read_pickle(os.path.join(dir_csv, fn))
print(df.columns)

## Resume after loading the saved csv file

In [ ]:
saveDir = r'Y:\Avinash\Projects\RS recruitment\Ablations\CSV files'
fn = ft.findAndSortFilesInDir(saveDir, ext = 'csv', search_str= 'mHomologs_intermediateRS_ventralRS')
if len(fn)>0:
    fn = fn[-1]
df = pd.read_csv(os.path.join(saveDir,fn))
df

In [ ]:
#%% Bend amplitude vs bend number

figDir = f'Y:\Avinash\Projects\RS recruitment\Figures\{util.timestamp()}'
if not os.path.exists(figDir):
    os.mkdir(figDir)
nBends = 10

# plt.style.use(('seaborn-paper', 'seaborn-whitegrid'))
amps = np.array(df.bendAmp_rel)
amps = np.delete(amps, np.where(np.isnan(amps)))
amp_low = np.percentile(amps,5)
amp_high = np.percentile(amps,99)

pers = np.array(df.bendInt)
pers = np.delete(pers, np.where(np.isnan(pers)))
per_low = np.percentile(pers,5)
per_high = np.percentile(pers,95)

df_now = df.loc[(df.bendAmp_rel>= amp_low) & (df.bendAmp_rel <= amp_high) & (df.bendInt >= per_low) & \
               (df.bendInt <= per_high) & (df.bendNum>=0) & (df.bendNum <=nBends)]

# df_now = df.loc[(df.bendAmp_rel>= 40) & (df.bendAmp_rel <= amp_high) & \
#                 (df.bendInt <= 60) & (df.bendNum>=0) & (df.bendNum <=10)]

plt.style.use(('seaborn-ticks', 'seaborn-paper'))
fh = sns.catplot(data = df_now, x = 'bendNum', y = 'bendAmp_rel', hue = 'treatment', col = 'ablationGroup',\
            col_order=['mHom', 'intermediateRS', 'ventralRS'], aspect = 1.5, dodge = True,\
            sharex=True, sharey=True, kind = 'point', palette=plt.cm.tab10(np.arange(10)),\
           hue_order = ['ctrl','abl'], height = 3, ci  =99, scale = 0.5)

figName = f'Fig_{util.timestamp()}_Bend amplitude vs bend number for ctrl and abl_mHom_intermediate_ventral'
fh.savefig(os.path.join(figDir, figName + '.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(figDir, figName + '.png'), format = 'png', dpi = 'figure')

print(f'Saved at {figDir}')

In [ ]:
#%% Bend interval
nBends = 10

df_now = df.loc[(df.bendAmp_rel>= amp_low) & (df.bendAmp_rel <= amp_high) & \
                (df.bendInt <= per_high) & (df.bendNum>=0) & (df.bendNum <=nBends)]


# plt.style.use(('seaborn-white', 'seaborn-paper'))
fh = sns.catplot(data = df_now, x = 'bendNum', y = 'bendInt', hue = 'treatment', col = 'ablationGroup',\
            col_order=['mHom', 'intermediateRS', 'ventralRS'], aspect = 1.5, dodge = True,\
            sharex=True, sharey=True, kind = 'point', palette=plt.cm.tab10(np.arange(10)),\
           hue_order = ['ctrl','abl'], height = 3, ci  =99, scale = 0.5)

figName = f'Fig_{util.timestamp()}_Bend interval vs bend number for ctrl and abl_mHom_intermediate_ventral'
fh.savefig(os.path.join(figDir, figName + '.pdf'), format = 'pdf', dpi = 'figure')
fh.savefig(os.path.join(figDir, figName + '.png'), format = 'png', dpi = 'figure')

print(f'Saved at {figDir}')

In [ ]:
df.columns